In [31]:
import sqlite3 as sql
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='bhavya311', api_key='gNNq8QAPGpyJ4RwOB5l2')
plotly.tools.set_config_file(world_readable=True,sharing='public')

con = sql.connect("../recipe.db")
con.row_factory = sql.Row
cur = con.cursor()

cur.execute("select distinct Cuisine from recipes")
rows = cur.fetchall()
for i in range(len(rows)):
    r = dict(rows[i])
    rows[i] = r

cuisines = [rows[i]['Cuisine'] for i in range(len(rows))]
con.close()
cuisines_titles = []
for cuisine in cuisines:
    con = sql.connect("../recipe.db")
    con.row_factory = sql.Row
    cur = con.cursor()
    cur.execute("Select * from recipes where Cuisine like '%" + cuisine + "%'")
    ings = cur.fetchall()
    con.close()
    for i in range(len(ings)):
        r = dict(ings[i])
        ings[i] = r
    ingredients = [ings[i]['Ingredients'] for i in range(len(ings))]
    count_ings = []
    for i in range(len(ingredients)):
        split_ings = ingredients[i].split('|')
        count_ings.append(len(split_ings))
    average_size = sum(count_ings)/len(count_ings)
    average_size = round(average_size, 3)
    sub_title = cuisine + " | Average Recipe Size: " + str(average_size)
    cuisines_titles.append(sub_title)
    
fig = plotly.tools.make_subplots(rows=(len(cuisines)+1)//3, cols=3, subplot_titles=tuple(cuisines_titles))
row = 1
col = 1
plot_no = 1

for cuisine in cuisines:
    con = sql.connect("../recipe.db")
    con.row_factory = sql.Row
    cur = con.cursor()
    cur.execute("select * from recipes where Cuisine like '%" + cuisine + "%'")
    ings = cur.fetchall()
    for i in range(len(ings)):
        r = dict(ings[i])
        ings[i] = r
    
    ingredients = [ings[i]['Ingredients'] for i in range(len(ings))]
    count_ings = []
    for i in range(len(ingredients)):
        split_ings = ingredients[i].split('|')
        count_ings.append(len(split_ings))
    
    print(count_ings)
    
#     for i in range(len(count_ings)):
#         count_ings[i] = 2.5 * round(count_ings[i] / 2.5)
    print(count_ings)
    d = {}
    
    for i in range(len(count_ings)):
        d[count_ings[i]] = 0
    for i in range(len(count_ings)):
        d[count_ings[i]] += 1
    
    ing = []
    percentage = []
    
    for c in d:
        d[c] = (float(d[c]) / len(count_ings)) * 100
        d[c] = round(d[c], 2)
        ing.append(c)
        percentage.append(d[c])
    
    trace = go.Scatter(x=ing, y=percentage,mode='markers+text', name = cuisine)
    fig.append_trace(trace, row, col)
    fig['layout']['xaxis'+str(plot_no)].update(title='Recipe Size', showgrid=False, mirror='ticks', showline=True)
    fig['layout']['yaxis'+str(plot_no)].update(title='Percentage', showgrid=False, mirror='ticks', showline=True)
    fig['layout'].update(showlegend=False)
    if col < 3:
        col += 1
    else:
        col = 1
        row += 1
    con.close()
    plot_no+=1

print(fig)

fig['layout'].update(height=6000, width=1300, title="Recipe Size of each Cuisine", titlefont=dict(family='Courier New, monospace', size=25, color='#7f7f7f'))

plotly.offline.plot(fig, filename='../templates/temp_recipe_size.html')


This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]   
[ (2,1) x4,y4 ]     [ (2,2) x5,y5 ]     [ (2,3) x6,y6 ]   
[ (3,1) x7,y7 ]     [ (3,2) x8,y8 ]     [ (3,3) x9,y9 ]   
[ (4,1) x10,y10 ]   [ (4,2) x11,y11 ]   [ (4,3) x12,y12 ] 
[ (5,1) x13,y13 ]   [ (5,2) x14,y14 ]   [ (5,3) x15,y15 ] 
[ (6,1) x16,y16 ]   [ (6,2) x17,y17 ]   [ (6,3) x18,y18 ] 
[ (7,1) x19,y19 ]   [ (7,2) x20,y20 ]   [ (7,3) x21,y21 ] 
[ (8,1) x22,y22 ]   [ (8,2) x23,y23 ]   [ (8,3) x24,y24 ] 
[ (9,1) x25,y25 ]   [ (9,2) x26,y26 ]   [ (9,3) x27,y27 ] 
[ (10,1) x28,y28 ]  [ (10,2) x29,y29 ]  [ (10,3) x30,y30 ]
[ (11,1) x31,y31 ]  [ (11,2) x32,y32 ]  [ (11,3) x33,y33 ]
[ (12,1) x34,y34 ]  [ (12,2) x35,y35 ]  [ (12,3) x36,y36 ]
[ (13,1) x37,y37 ]  [ (13,2) x38,y38 ]  [ (13,3) x39,y39 ]
[ (14,1) x40,y40 ]  [ (14,2) x41,y41 ]  [ (14,3) x42,y42 ]
[ (15,1) x43,y43 ]  [ (15,2) x44,y44 ]  [ (15,3) x45,y45 ]
[ (16,1) x46,y46 ]  [ (16,2) x47,y47 ]  [ (16,3) x48,y48 ]
[ (17,1) x49,y49 ]

[10, 10, 6, 9, 10, 16, 8, 14, 13, 16, 12, 10, 15, 8, 8, 9, 12, 6, 9, 12, 8, 8, 13, 11, 14, 9, 7, 9, 10, 12, 10, 14, 8, 6, 12, 11, 7, 6, 8, 13, 15, 6, 15, 8, 12, 8, 13, 15, 9, 6, 10, 12, 12, 8, 12, 11, 9, 8, 11, 9, 3, 10, 10, 14, 10, 10, 11, 7, 10, 12, 11, 13, 15, 13, 12, 20, 11, 7, 5, 17, 10, 8, 8, 13, 14, 6, 13, 9, 11, 17, 7, 9, 12, 12, 5, 17, 11, 18, 14, 12, 7, 8, 16, 10, 7, 10, 9, 8, 10, 12, 8, 10, 15, 7, 10, 10, 7, 20, 11, 17, 17, 6, 13, 9]
[10, 10, 6, 9, 10, 16, 8, 14, 13, 16, 12, 10, 15, 8, 8, 9, 12, 6, 9, 12, 8, 8, 13, 11, 14, 9, 7, 9, 10, 12, 10, 14, 8, 6, 12, 11, 7, 6, 8, 13, 15, 6, 15, 8, 12, 8, 13, 15, 9, 6, 10, 12, 12, 8, 12, 11, 9, 8, 11, 9, 3, 10, 10, 14, 10, 10, 11, 7, 10, 12, 11, 13, 15, 13, 12, 20, 11, 7, 5, 17, 10, 8, 8, 13, 14, 6, 13, 9, 11, 17, 7, 9, 12, 12, 5, 17, 11, 18, 14, 12, 7, 8, 16, 10, 7, 10, 9, 8, 10, 12, 8, 10, 15, 7, 10, 10, 7, 20, 11, 17, 17, 6, 13, 9]
[17, 10, 17, 21, 22, 7, 9, 9, 12, 8, 12, 15, 14, 17, 11, 8, 12, 13, 9, 11, 7, 14, 16, 15, 11, 21, 12, 

[7, 14, 7, 11, 10, 5, 12, 15, 4, 18, 9, 11, 10, 6, 8, 9, 9, 13, 30, 17, 11, 10, 6, 5, 7, 10, 10, 12, 4, 8, 13, 12, 10, 5, 10, 3, 10, 10, 6, 12, 9, 13, 7, 10]
[7, 14, 7, 11, 10, 5, 12, 15, 4, 18, 9, 11, 10, 6, 8, 9, 9, 13, 30, 17, 11, 10, 6, 5, 7, 10, 10, 12, 4, 8, 13, 12, 10, 5, 10, 3, 10, 10, 6, 12, 9, 13, 7, 10]
[6, 16, 8, 11, 7, 7, 9, 5, 6, 13, 8, 12, 8, 15, 8, 13, 3, 8, 17, 6, 23, 6, 8, 14, 11, 18, 7, 6, 14, 5, 8, 12, 14, 4, 9, 9, 5, 6, 8, 10, 17, 6, 11, 9, 6, 10, 16, 14, 11, 6, 5, 14, 12, 8, 12, 7, 12, 6, 9, 6, 13, 7, 8, 8, 15, 11, 6, 9, 5, 6, 10, 10, 6, 15, 8, 10, 5, 8, 10, 15, 9, 3, 8, 7, 2, 11, 12, 12, 7, 12, 5, 9, 10, 6, 18, 15, 14, 12, 8, 22, 12, 12, 10, 10]
[6, 16, 8, 11, 7, 7, 9, 5, 6, 13, 8, 12, 8, 15, 8, 13, 3, 8, 17, 6, 23, 6, 8, 14, 11, 18, 7, 6, 14, 5, 8, 12, 14, 4, 9, 9, 5, 6, 8, 10, 17, 6, 11, 9, 6, 10, 16, 14, 11, 6, 5, 14, 12, 8, 12, 7, 12, 6, 9, 6, 13, 7, 8, 8, 15, 11, 6, 9, 5, 6, 10, 10, 6, 15, 8, 10, 5, 8, 10, 15, 9, 3, 8, 7, 2, 11, 12, 12, 7, 12, 5, 9, 10, 6, 1

[9, 2, 13, 6, 5, 16, 5, 9, 4, 7, 7, 10, 12, 14, 15, 10, 5, 4, 2, 14, 17, 8, 12, 15, 12, 1, 19, 12, 9, 19, 9, 3, 21, 10, 18, 12, 12, 12, 7, 10, 17, 5, 4, 10, 4, 11, 11, 11, 11, 16, 10, 7, 20, 12, 14, 24, 6, 9, 13, 6, 13, 6, 9, 9, 16, 11, 8, 10, 6, 8, 7, 6, 4, 14, 10, 6, 9, 12, 12, 13, 5, 12, 7, 11, 9, 5, 16, 8, 4, 18, 6, 9, 10, 3, 9, 13, 8, 13, 9, 8, 7, 8, 6, 7, 15, 7, 11, 11, 14, 11, 8, 16, 7, 8, 7, 17, 6, 9, 7, 5, 15, 17, 12, 9, 5, 7, 12, 18, 13, 6, 9, 15, 4, 4, 30, 20, 14, 2, 6, 5, 7, 18, 10, 12, 14, 6, 3, 6, 10, 12, 12, 5, 6, 13, 5, 11, 9, 7, 11, 8, 8, 13, 9, 6, 7, 8, 17, 20, 3, 14, 8, 4, 6, 6, 4, 7, 10, 3, 10, 12, 10, 7, 5, 8, 4, 6, 6, 14, 10, 5, 6, 5, 7, 4, 8, 12, 11, 4, 4, 3, 9, 7, 7, 6, 7, 3, 11, 6, 8, 6, 7, 6, 5, 10, 9, 12, 10, 12, 9, 9, 10, 10, 8, 9, 9, 6, 8, 3, 6, 7, 6, 5, 13, 6, 21, 19, 22, 9, 7, 11, 17, 14, 7, 4, 15, 8, 9, 7, 15, 8, 3, 6, 8, 9, 15, 8, 7, 15, 12, 14, 3, 14, 6, 5, 9, 8, 10, 4, 8, 8, 11, 5, 10, 10, 8, 10, 13, 6, 18, 18, 11, 5, 12, 7, 10, 5, 7, 9, 16, 11, 10, 1

[10, 8, 13, 21, 9, 9, 6, 11, 19, 16, 17, 17, 11, 13, 16, 10, 8, 14, 17, 8, 11, 8, 11, 11, 9, 20, 17, 5, 7, 11, 10, 15, 11, 14, 24, 12, 13, 15, 20, 16, 5, 10, 16, 22, 14, 9, 9, 14, 14, 6, 23, 15, 19]
[10, 8, 13, 21, 9, 9, 6, 11, 19, 16, 17, 17, 11, 13, 16, 10, 8, 14, 17, 8, 11, 8, 11, 11, 9, 20, 17, 5, 7, 11, 10, 15, 11, 14, 24, 12, 13, 15, 20, 16, 5, 10, 16, 22, 14, 9, 9, 14, 14, 6, 23, 15, 19]
[12, 5, 9, 9, 11]
[12, 5, 9, 9, 11]
[9, 14, 13, 6, 8, 17, 14, 15, 15, 19, 14, 11, 18, 23, 15, 18, 12, 5, 10, 10, 15, 18, 24, 10, 12, 10, 10, 12, 20, 9, 18, 13, 16, 16, 15, 9, 15, 16, 10, 10, 16, 9, 19, 12, 11, 12, 22, 17, 19, 20]
[9, 14, 13, 6, 8, 17, 14, 15, 15, 19, 14, 11, 18, 23, 15, 18, 12, 5, 10, 10, 15, 18, 24, 10, 12, 10, 10, 12, 20, 9, 18, 13, 16, 16, 15, 9, 15, 16, 10, 10, 16, 9, 19, 12, 11, 12, 22, 17, 19, 20]
[10, 15, 10, 14, 5, 11, 15, 10, 9, 12, 8, 12, 14, 9, 17, 11, 15, 12, 14, 14]
[10, 15, 10, 14, 5, 11, 15, 10, 9, 12, 8, 12, 14, 9, 17, 11, 15, 12, 14, 14]
[14, 8, 13, 7, 11, 16, 1

Figure({
    'data': [{'mode': 'markers+text',
              'name': 'Japanese',
              'type': 'scatter',
              'uid': '6c214d70-da7d-4c9a-9e83-86da20ab0842',
              'x': [10, 7, 5, 9, 3, 16, 6, 11, 13, 12, 8, 21, 4, 14, 15, 23, 30,
                    22, 17, 2],
              'xaxis': 'x',
              'y': [6.36, 14.41, 5.08, 11.44, 4.24, 2.54, 11.02, 6.78, 8.05, 5.93,
                    10.59, 0.42, 4.66, 2.97, 2.54, 0.42, 0.42, 0.85, 0.42, 0.85],
              'yaxis': 'y'},
             {'mode': 'markers+text',
              'name': 'Chinese',
              'type': 'scatter',
              'uid': '06402cfd-cc76-4fb2-81be-b3dff6cc1f57',
              'x': [16, 12, 14, 15, 10, 9, 13, 7, 21, 18, 11, 6, 8, 17, 24, 19, 5,
                    4, 3, 23],
              'xaxis': 'x2',
              'y': [3.52, 10.56, 5.63, 6.69, 10.92, 10.21, 7.39, 6.34, 1.41, 3.17,
                    8.45, 3.87, 8.8, 1.76, 0.35, 1.41, 4.23, 2.46, 2.11, 0.7],
              'yaxis

'../templates/temp_recipe_size.html'